In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import re
import requests
import math
import time
import numpy as np
from tqdm import tqdm

In [3]:
BASE_URL = "https://www.kjnala.com"
search_url = f'{BASE_URL}/board/index.jsp?code=bbs018&page=1'
page = requests.get(search_url)
soup = BeautifulSoup(page.content, 'html.parser')

num = soup.select('td.pc_line')[0].text
end_num = math.ceil(int(num) / 10)

df = pd.read_csv('./통합.csv')
stop_date = df['date'].iloc[0]

In [4]:
def get_items(date, dtype, link):
    search_url = f'{BASE_URL}{link}'
    page = requests.get(search_url)
    soup = BeautifulSoup(page.content, 'html.parser')

    items = soup.select('div.contArea > p')
    
    # Filter items with more than 9 characters
    filtered_items = [item for item in items if len(item.text) >= 9]

    # Reassign items with filtered items
    items = filtered_items
    
    count = str(items).count('<p>')
    
    item_list = []
    
    for i in range(len(items)):
        
        if '<br/>' in str(items[i]):
            item_t = re.sub('<p>', '', str(items[i]))
            item_t = re.sub('</p>', '', item_t)
            item_t = item_t.replace(u'\xa0', u'')
            item_t = item_t.split('<br/>')
            while("" in item_t):
                item_t.remove("")

            item_t =[s[1:].lstrip() if s.startswith('-') else s.lstrip('- ').lstrip() for s in item_t]
            
            if count > 3:
                item_t = [''.join(item_t)]
            
            for i in range(len(item_t)):
                if '-' in item_t[i]:
                    item = item_t[i].split('-', 1)[0]
                    item2 = item_t[i].split('-', 1)[1]
                    item_list.append([date, item.rstrip(), item2.lstrip(), dtype])
                elif ' ' in item_t[i]:
                    item = item_t[i].split(' ', 1)[0]
                    item2 = item_t[i].split(' ', 1)[1]
                    item_list.append([date, item.rstrip(), item2.lstrip(), dtype])
                else: 
                    item_list.append([date, item_t[i], np.nan, dtype])

    if item_list == []:
        for i in range(len(items)):
            item_t = re.sub('<p>', '', str(items[i]))
            item_t = re.sub('</p>', '', item_t)
            item_t = item_t.replace(u'\xa0', u'')
                    
            if '-' in item_t:
                item = item_t.split('-', 1)[0]
                item2 = item_t.split('-', 1)[1]
                item_list.append([date, item.rstrip(), item2.lstrip(), dtype])
            elif ' ' in item_t:
                item = item_t.split(' ', 1)[0]
                item2 = item_t.split(' ', 1)[1]
                item_list.append([date, item.rstrip(), item2.lstrip(), dtype])
            else: 
                item_list.append([date, item_t[i], np.nan, dtype])
    
    return item_list

In [ ]:
data = []
def get_link(page):
    search_url = f'{BASE_URL}/board/index.jsp?code=bbs018&page={page}'
    page = requests.get(search_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    global stop_reached  # Use global flag
    
    nname = soup.select('p.subject > a')
    
    for i in range(len(nname)):
        title = re.sub('\s+', '', nname[i].text)
        if '사위' in title:
            dtype = '사위일체'
        elif '효자' in title:
            dtype = '효자종목'
        elif '저점' in title:
            dtype = '저점장대양봉'
        else:
            continue
        number = re.findall("\d+", title)
        date = '-'.join(number)[0:10]
        
        if date == stop_date:
            stop_reached = True  # Set the flag to indicate stop_date is reached
            return data  # Exit the function
        
        # Skip get_items if the length of the date string is less than 5
        if len(date) < 5:
            continue
        
        # 혹여나 코드로 인해 해당 페이지가 안보일 경우
        #link = re.sub('code=bbs018&', '', nname[i]['href'])
        
        link = nname[i]['href']
        try: 
            data.extend(get_items(date, dtype, link))
            
        except:
            print(date)
    return data

In [ ]:
stop_reached = False  # Flag to track stop_date condition
for i in tqdm(range(1, end_num+1)): 
    data = get_link(i)
    if stop_reached:  # Check if stop_date is reached inside get_link
        break  # Break out of the loop if the stop_date is reached
    time.sleep(2)

In [5]:
search_url = f'{BASE_URL}/board/index.jsp?code=bbs018&page={1}'
page = requests.get(search_url)
soup = BeautifulSoup(page.content, 'html.parser')


nname = soup.select('p.subject > a')

for i in range(len(nname)):
    title = re.sub('\s+', '', nname[i].text)
    if '사위' in title:
        dtype = '사위일체'
    elif '효자' in title:
        dtype = '효자종목'
    elif '저점' in title:
        dtype = '저점장대양봉'
    else:
        continue
    number = re.findall("\d+", title)
    date = '-'.join(number)[0:10]
    
    if date == stop_date:
        stop_reached = True  # Set the flag to indicate stop_date is reached
        continue
    
    # Skip get_items if the length of the date string is less than 5
    if len(date) < 5:
        continue
    
    # 혹여나 코드로 인해 해당 페이지가 안보일 경우
    #link = re.sub('code=bbs018&', '', nname[i]['href'])
    
    link = nname[i]['href']
    print(date, dtype, link)

2024-02-01 저점장대양봉 /board/read.jsp?id=1852426&code=bbs018&page=1
2024-02-01 효자종목 /board/read.jsp?id=1852423&code=bbs018&page=1
2024-02-01 사위일체 /board/read.jsp?id=1852422&code=bbs018&page=1
2024-01-30 저점장대양봉 /board/read.jsp?id=1852041&code=bbs018&page=1


In [11]:
search_url = f'{BASE_URL}/board/read.jsp?id=1852426&code=bbs018'
page = requests.get(search_url)
soup = BeautifulSoup(page.content, 'html.parser')

soup


<script>alert('로그인이 필요한 서비스입니다.');</script><script>window.location.replace('/member/login.jsp?returl=/board/read.jsp?id=1852426&code=bbs018');</script>

In [ ]:
import requests

# Login credentials
username = 'stanfm'
password = 'Bee4answer*'

# Login endpoint
login_url = 'https://www.kjnala.com/member/login.jsp'

# Page to scrape after login
scrape_url = 'https://www.kjnala.com/board/read.jsp?id=1852426&code=bbs018&page=1'

# Create a session
session = requests.Session()

# Perform login
login_data = {
    'id': username,
    'passwd': password
}
response = session.post(login_url, data=login_data)

scrape_response = session.get(scrape_url)

soup = BeautifulSoup(scrape_response.text, 'html.parser')
soup


In [38]:
search_url = f'{BASE_URL}{link}'
scrape_response = session.get(search_url)
soup = BeautifulSoup(scrape_response.text, 'html.parser')

items = soup.select('div.contArea > p')

# Filter items with more than 9 characters
filtered_items = [item for item in items if len(item.text) >= 9]

# Reassign items with filtered items
items = filtered_items

count = str(items).count('<p>')

item_list = []

for i in range(len(items)):
    if '<br/>' in str(items[i]) or '<br>' in str(items[i]):
        item_t = re.sub('<p>', '', str(items[i]))
        item_t = re.sub('</p>', '', item_t)
        item_t = item_t.replace(u'\xa0', u'')
        item_t = re.split(r'<br/>|<br>', item_t)
        while("" in item_t):
            item_t.remove("")

        item_t =[s[1:].lstrip() if s.startswith('-') else s.lstrip('- ').lstrip() for s in item_t]
        
        if count > 3:
            item_t = [''.join(item_t)]
        
        for i in range(len(item_t)):
            if '-' in item_t[i]:
                item = item_t[i].split('-', 1)[0]
                item2 = item_t[i].split('-', 1)[1]
                item_list.append([date, item.rstrip(), item2.lstrip(), dtype])
            elif ' ' in item_t[i]:
                item = item_t[i].split(' ', 1)[0]
                item2 = item_t[i].split(' ', 1)[1]
                item_list.append([date, item.rstrip(), item2.lstrip(), dtype])
            else: 
                item_list.append([date, item_t[i], np.nan, dtype])

if item_list == []:
    for i in range(len(items)):
        item_t = re.sub('<p>', '', str(items[i]))
        item_t = re.sub('</p>', '', item_t)
        item_t = item_t.replace(u'\xa0', u'')
                
        if '-' in item_t:
            item = item_t.split('-', 1)[0]
            item2 = item_t.split('-', 1)[1]
            item_list.append([date, item.rstrip(), item2.lstrip(), dtype])
        elif ' ' in item_t:
            item = item_t.split(' ', 1)[0]
            item2 = item_t.split(' ', 1)[1]
            item_list.append([date, item.rstrip(), item2.lstrip(), dtype])
        else: 
            item_list.append([date, item_t[i], np.nan, dtype])
    
print(item_list)

[['2024-01-30', 'KG ETS', '도는 것 같다', '저점장대양봉'],
 ['2024-01-30', '동국씨엠', '좋은 움직임을 보여주는 거 같다', '저점장대양봉'],
 ['2024-01-30', '유진투자증권', '4000원 넘으면 다중봉 다중턱이 기다리고 있다', '저점장대양봉'],
 ['2024-01-30', '동양생명', '첩첩산중', '저점장대양봉'],
 ['2024-01-30', '피엠티', '아직 원매수를 털기엔 살짝 부족', '저점장대양봉'],
 ['2024-01-30', '삼성엔지니어링', '어제 떨어진거에 비해 음봉벽타기', '저점장대양봉'],
 ['2024-01-30', '다우데이타', '피하는게 좋다', '저점장대양봉'],
 ['2024-01-30', '쌍용C&amp;E', '월봉이 상대적으로 높다', '저점장대양봉'],
 ['2024-01-30', '메드팩토', '내리막폭포가 마음에 걸린다', '저점장대양봉'],
 ['2024-01-30', '오킨스전자', '다시 음봉벽타기를 하고 있다', '저점장대양봉'],
 ['2024-01-30', '미래에셋증권', '조금 더 가는 것 같다', '저점장대양봉'],
 ['2024-01-30', '삼성물산', '600일 신고가를 썼다', '저점장대양봉'],
 ['2024-01-30', '한국비엔씨', '슬슬 기지개를 피지 않을까', '저점장대양봉'],
 ['2024-01-30', '한국가스공사', '대형주들이 움직이니까 같이 움직이지 않나', '저점장대양봉'],
 ['2024-01-30', '휴맥스홀딩스', '반등하려고 애를 쓰고 있다', '저점장대양봉'],
 ['2024-01-30', '현대차', '워낙무겁다', '저점장대양봉'],
 ['2024-01-30', 'NH투자증권', '오늘 5프로 이상 상승하는데 이런 적이 거의 없는 무거운 종목이다', '저점장대양봉'],
 ['2024-01-30', 'DGB금융지주', '올라갈수록 첩첩산중이다 하도 물린 사람들이 많아서', '저점장대양봉

: 